<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/weinstein_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Stock Screening Inspired by Stan Weinstein

This notebook features stock screening methods inspired by Stan Weinstein's book, *Secrets for Profiting in Bull and Bear Markets*. The strategies and criteria used in this analysis are based on the principles outlined in Weinstein's work, aiming to identify potential investment opportunities in both bull and bear markets.


### Install and Setup (this section will be executed automatically)

#### Install Required Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git to /tmp/pip-req-build-1_pipcoq
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-1_pipcoq
  Resolved https://github.com/yorkjong/vistock.git to commit 9a1c297ff71e5928d2c695ba72c4144d1228db05
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.5 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.5.2-py3-none-any.whl size=81954 sha256=c194a4abdabaa57e5ec9775b19038a30adb492da36dbdf2b8c851bed8a439f8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-wy7l6rld/wheels/1f/ff/6c/9ec0adab3f1af71ff9ba3e4eb52d0f3849d3a695b812dc509a
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.5 MB/s eta 0:00:00
--2024-09-09 13:34:56--  https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?

In [2]:
# @title Install Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

#### Setup and Configuration

In [3]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [4]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [5]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    dir='weinstein'
)

In [6]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [7]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return list(df['Name'])
    elif 'Ticker' in df.columns:
        return list(df['Ticker'])
    return []

def sorting_tickers():
    return list(tickers_from_df(df_top_f1))

def increasing_rs_tickers():
    return list(tickers_from_df(df_top_f2))

def rs_breakout_tickers():
    return list(tickers_from_df(df_top_f3))

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

In [8]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

In [9]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [27]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [10]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The Source of Stocks to Analyze)
- **Description**: Stocks can be sourced from various exchanges or indices.
- **Common Abbreviations**:
  - **Taiwan Markets**:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所)
    - `TPEX`: Taipei Exchange (上櫃交易所)
    - `ESB`: Emerging Stock Board (興櫃交易所)
    - Combine with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - **America Markets**:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index (費半指數)
    - Combine with `+` (e.g., `SPX+DJIA+NDX+SOX`)

#### Period (Historical Data Time Range)
- **Description**: The duration for fetching historical data.
- **Example**: `2y` for 2 years

#### Interval (Historical Data Frequency)
- **Description**: The frequency of historical data points.
- **Common Intervals**:
  - `'1d'`: Daily data
  - `'1wk'`: Weekly data
  - `'1mo'`: Monthly data

#### MA (Moving Average)
- **SMA**: Simple Moving Average
- **EMA**: Exponential Moving Average

#### RS (Relative Strength)
- **Description**: Measures a stock's performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the performance of the benchmark index or market.
  - A positive value indicates outperformance relative to the benchmark.
  - A negative value indicates underperformance relative to the benchmark.

#### EPS RS (EPS Relative Strength)
- **Description**: Measures a stock's earnings per share (EPS) performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the EPS performance of the benchmark index.
  - A positive value indicates that the stock's EPS growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index EPS Calculation**: Weighted according to component stocks' share counts.

#### Rev RS (Operating Revenue Relative Strength)
- **Description**: Measures a stock's operating revenue performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the revenue performance of the benchmark index.
  - A positive value indicates that the stock's revenue growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index Revenue Calculation**: Weighted according to component stocks' market capitalization.

#### TTM EPS (Trailing Twelve Months EPS)
- **Description**: The earnings per share over the trailing twelve months, which gives a more recent view of the company's profitability.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM EPS indicates better profitability over the past year.
  - Used to compare with previous periods or with other companies.

#### TTM RPS (Trailing Twelve Months Revenue Per Share)
- **Description**: The revenue per share over the trailing twelve months, showing how much revenue a company generates per share.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM RPS indicates better revenue generation capability per share over the past year.
  - Useful for comparing revenue performance with previous periods or with other companies.

#### TTM PE (Trailing Twelve Months PE)
- **Description**: The price-to-earnings ratio calculated using the earnings from the trailing twelve months. It provides a measure of how much investors are willing to pay per dollar of earnings.
- **Unit**: Ratio
- **Interpretation**:
  - A higher TTM PE indicates that investors expect higher future growth and are willing to pay more for the stock.
  - A lower TTM PE might suggest undervaluation or lower expected growth.


##### References:
- [Mansfield Relative Strength (Original Version) by stageanalysis — Indicator by Stage_Analysis — TradingView](https://www.tradingview.com/script/NzUBDDtb-Mansfield-Relative-Strength-Original-Version-by-stageanalysis/)
- [Mansfield Relative Strength | TrendSpider Store](https://trendspider.com/trading-tools-store/indicators/mansfield-relative-strength/)
- [Mansfield Relative Strength | ChartMill.com](https://www.chartmill.com/documentation/technical-analysis/indicators/35-Mansfield-Relative-Strength)
- [How to create the Mansfield Relative Performance Indicator - Stage Analysis](https://www.stageanalysis.net/blog/4266/how-to-create-the-mansfield-relative-performance-indicator)
  - [Stan Weinstein's Stage Analysis | Page 49 | Trade2Win Forums • UK Financial Trading Community](https://www.trade2win.com/threads/stan-weinsteins-stage-analysis.134944/page-49#post-2137398)


## Execute Actions Step by Step


In [11]:
# @title Step 1. Pick a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[4].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames), reverse=True)
all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
all_periods = sorted(set(fn.split('_')[2] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames to set dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=all_types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

period_dropdown = widgets.Dropdown(
    options=all_periods,
    description='Period:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files), reverse=True)
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update Type dropdown
    available_types = sorted(set(fn.split('_')[1] for fn in date_filtered_files), reverse=True)
    type_dropdown.options = available_types
    if type_dropdown.value not in available_types:
        type_dropdown.value = available_types[0] if available_types else None

    # Update Period dropdown
    available_periods = sorted(set(fn.split('_')[2] for fn in date_filtered_files), reverse=True)
    period_dropdown.options = available_periods
    if period_dropdown.value not in available_periods:
        period_dropdown.value = available_periods[0] if available_periods else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            type_dropdown.value == fn.split('_')[1] and
            period_dropdown.value == fn.split('_')[2])
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')
period_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, period_dropdown, file_dropdown)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='510px'), options=('20240909', '20240908'), value='20240909'…

Dropdown(description='Source:', layout=Layout(width='510px'), options=('TWSE+TPEX', 'SPX+DJIA+NDX+RUI+SOX', 'S…

Dropdown(description='Type:', layout=Layout(width='510px'), options=('stocks',), value='stocks')

Dropdown(description='Period:', layout=Layout(width='510px'), options=('2y',), value='2y')

Dropdown(description='File:', layout=Layout(width='510px'), options=(), value=None)

### Step 2. Filter Stocks

In [12]:
# @title Filter 1. Sorting {"run":"auto"}
cond0 = "(df['RS (%)'] > 0)" # @param ["(df['RS (%)'] > 0)"]
cond1 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond2 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond3 = "& (df['Rev RS (%)'] > 0)" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond4 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond5 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
sorted_column = "RS (%)" # @param ["RS (%)","1 Week Ago","1 Month Ago","3 Months Ago","6 Months Ago","Price","MA10","MA30","Volume / VMA10","EPS RS (%)", "Rev RS (%)"]
ascending = False # @param {"type":"boolean"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 6
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, ma, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nMA: {ma}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 2y
MA: SMA
Date: 20240909


,Ticker,Name,Sector,Industry,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,RS Rank (%),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
12,2365,昆盈,Technology,Computer Hardware,105.73,107.25,166.21,96.53,75.81,2.80,99.35,58.20,61.07,37.16,0.43,6.32,0.45,6.17,4.094,131.11
13,8937,合騏,Consumer Cyclical,Recreational Vehicles,104.32,130.26,50.14,-9.71,-10.90,4.72,99.30,57.20,43.56,28.85,1.39,46.78,1.12,46.77,1.649,50.54
15,1540,喬福,Industrials,Specialty Industrial Machinery,101.00,72.66,66.59,-0.56,-7.91,-3.61,99.19,47.70,36.28,25.19,2.27,28.99,2.09,15.58,7.516,21.94
16,3535,晶彩科,Technology,Computer Hardware,99.70,112.59,146.43,23.17,3.33,-13.74,99.13,67.10,70.56,40.90,0.43,144.09,1.28,127.32,8.837,53.75
22,2543,皇昌,Industrials,Engineering & Construction,89.94,101.38,81.54,97.86,142.98,182.84,98.81,71.90,64.04,48.23,0.60,177.84,4.99,13.70,46.896,14.33
28,6609,瀧澤科,Technology,Electronic Components,79.80,72.30,22.81,28.06,-12.74,-17.38,98.49,75.20,56.62,43.90,1.65,31.38,2.45,12.55,32.536,30.20
29,3284,太普高,Industrials,Specialty Industrial Machinery,79.01,107.83,89.98,42.94,15.88,-0.20,98.43,53.30,50.64,36.31,0.79,74.45,1.65,22.14,10.217,31.58
36,4561,健椿,Industrials,Metal Fabrication,74.34,74.28,18.55,35.56,10.18,29.72,98.05,77.40,59.89,47.93,2.46,6.08,4.28,7.94,22.745,17.76
37,3379,彬台,Industrials,Specialty Industrial Machinery,72.49,79.13,117.33,30.32,-0.24,6.35,98.00,38.45,41.45,25.41,0.43,340.48,0.48,38.78,18.121,80.21
46,6215,和椿,Industrials,Specialty Industrial Machinery,66.84,57.62,61.03,65.85,0.43,-9.74,97.51,69.90,64.87,49.94,1.73,64.79,1.27,15.28,13.352,53.46


昆盈, 合騏, 喬福, 晶彩科, 皇昌, 瀧澤科, 太普高, 健椿, 彬台, 和椿
2365, 8937, 1540, 3535, 2543, 6609, 3284, 4561, 3379, 6215


In [13]:
# @title Filter 2. Increasing RS > 0 {"run":"auto"}
cond0 = "(df['RS (%)'] > 0)" # @param ["(df['RS (%)'] > 0)"]
cond1 = "& (df['RS (%)'] > df['1 Week Ago'])" # @param ["& (df['RS (%)'] > df['1 Week Ago'])", ""]
cond2 = "& (df['1 Week Ago'] > df['1 Month Ago'])" # @param ["& (df['1 Week Ago'] > df['1 Month Ago'])", "& (df['RS (%)'] > df['1 Month Ago'])", ""]
cond3 = "& (df['1 Month Ago'] > df['3 Months Ago'])" # @param ["& (df['1 Month Ago'] > df['3 Months Ago'])", ""]
cond4 = "& (df['3 Months Ago'] > df['6 Months Ago'])" # @param ["& (df['3 Months Ago'] > df['6 Months Ago'])", ""]
cond5 = "& (df['6 Months Ago'] > df['9 Months Ago'])" # @param ["& (df['6 Months Ago'] > df['9 Months Ago'])", ""]
cond6 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond7 = "" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond8 = "" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond9 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond10 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 11
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, ma, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nMA: {ma}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df_top_f2 = df.head(num_items)
display(df_top_f2)

print_column(df_top_f2, 'Name')
print_column(df_top_f2, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 2y
MA: SMA
Date: 20240909


,Ticker,Name,Sector,Industry,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,RS Rank (%),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
104,2504,國產,Basic Materials,Building Materials,38.79,32.18,29.69,27.82,9.76,-6.02,94.38,56.00,50.18,44.95,0.94,72.96,3.99,-7.55,18.248,13.91
179,2495,普安,Technology,Computer Hardware,22.46,16.55,14.59,11.20,0.53,-6.58,90.32,32.05,31.68,26.65,1.88,2.18,1.57,-18.97,5.117,21.18
282,8021,尖點,Industrials,Tools & Accessories,12.57,9.88,2.41,-9.16,-11.82,-11.89,84.75,37.30,36.02,31.96,0.81,72.32,1.12,3.12,22.125,32.41
333,2889,國票金,Financial Services,Capital Markets,8.96,4.68,4.32,2.60,-9.44,-11.28,81.99,15.80,15.57,14.48,0.80,-3.35,0.59,0.46,2.136,26.27
339,3597,映興,Technology,Electronic Components,8.44,-2.14,-6.59,-10.30,-11.61,-19.67,81.67,28.40,26.38,24.80,0.61,-7.96,0.24,0.70,14.650,125.00
401,2015,豐興,Basic Materials,Steel,5.51,1.70,-0.62,-9.28,-9.89,-9.98,78.34,82.70,80.29,74.32,1.27,17.51,4.35,-5.97,59.582,18.48
402,1708,東鹼,Basic Materials,Chemicals,5.48,2.59,-5.82,-15.67,-19.87,-20.42,78.26,38.40,36.30,33.56,0.58,46.45,2.77,-0.71,23.793,14.10
407,5603,陸海,Industrials,Integrated Freight & Logistics,5.28,1.24,-0.18,-4.63,-5.05,-9.39,77.99,26.85,26.32,24.47,0.68,38.24,2.22,17.79,16.939,12.14


國產, 普安, 尖點, 國票金, 映興, 豐興, 東鹼, 陸海
2504, 2495, 8021, 2889, 3597, 2015, 1708, 5603


In [26]:
# @title Filter 3. RS Breakout {"run":"auto"}
base = 0 # @param ["0","30","60","100"] {"type":"raw"}
cond0 = "(df['RS (%)'] > base)" # @param ["(df['RS (%)'] > base)"]
cond1 = "" # @param ["& (df['1 Week Ago'] < base)",""]
cond2 = "& (df['1 Month Ago'] < base)" # @param ["& (df['1 Month Ago'] < base)",""]
cond3 = "& (df['3 Months Ago'] < base)" # @param ["& (df['3 Months Ago'] < base)",""]
cond4 = "& (df['6 Months Ago'] < base)" # @param ["& (df['6 Months Ago'] < base)",""]
cond5 = "& (df['9 Months Ago'] < base)" # @param ["& (df['9 Months Ago'] < base)",""]
cond6 = "& (df['Volume / VMA10'] > 2)" # @param ["& (df['Volume / VMA10'] > 1.5)", "& (df['Volume / VMA10'] > 2)", "& (df['Volume / VMA10'] > 3)", "& (df['Volume / VMA10'] > 4)", ""]
cond7 = "& (df['Price'] > df['MA30'])" # @param ["& (df['Price'] > df['MA10'])", "& (df['Price'] > df['MA30'])", ""]
cond8 = "" # @param ["& (df['MA10'] > df['MA30'])",""]
cond9 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond10 = "" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond11 = "" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond12 = "" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond13 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", "& (df['Price'] < 500)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 14
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, ma, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nMA: {ma}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df_top_f3 = df.head(num_items)
display(df_top_f3)

print_column(df_top_f3, 'Name')
print_column(df_top_f3, 'Ticker')

Source: SPX+DJIA+NDX+RUI+RUT+SOX
Type: stocks
Period: 2y
MA: SMA
Date: 20240909


,Ticker,Sector,Industry,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,RS Rank (%),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
387,SMAR,Technology,Software—Application,11.6,-0.23,-1.0,-20.69,-13.53,-2.71,84.37,49.35,47.66,42.77,2.29,152.58,-0.32,1.55,7.615,NaN


SMAR


### Step 3. Visualize Filtered Stocks

In [17]:
# @title Plot 1. Mansfield Relative Strength Comparison {"run":"auto"}
source = "Filter 3. RS Breakout" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Major Global Stock Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1. Sorting': sorting_tickers,
    'Filter 2. Increasing RS': increasing_rs_tickers,
    'Filter 3. RS Breakout': rs_breakout_tickers,
    'Major Global Stock Indices': major_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
period_dropdown = create_period_dropdown()
interval_dropown = create_interval_dropdown()
cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'checkers'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
button = widgets.Button(description="Generate Plot")
out_msg, out_fig = widgets.Output(), widgets.Output()

menus = widgets.VBox([period_dropdown, interval_dropown, cmp_theme_dropdown])
display(checkbox_grid, menus, button, out_msg, out_fig)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

rs_cmp = {
    'mplfinance': mpl_mansfield.RelativeStrengthLines,
    'Plotly': ply_mansfield.RelativeStrengthLines,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_button_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        if rs_cmp is mpl_mansfield.RelativeStrengthLines:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        style=cmp_theme_dropdown.value)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        template=cmp_theme_dropdown.value)

button.on_click(on_button_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


GridBox(children=(Checkbox(value=True, description='迎輝', layout=Layout(width='auto'), style=DescriptionStyle(d…

Button(description='Generate Plot', style=ButtonStyle())

Output()

Output()

In [48]:
# @title Plot 2. Mansfield Stock Chart {"run":"auto"}
source = "U.S. Stocks" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Major Global Stock Indices", "U.S. Stocks", "Taiwan Stocks"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import functools as ft
import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf

import vistock.stock_indices as si
import vistock.tw as tw

symbols = {
    'Filter 1. Sorting': sorting_tickers,
    'Filter 2. Increasing RS': increasing_rs_tickers,
    'Filter 3. RS Breakout': rs_breakout_tickers,
    'Major Global Stock Indices': major_indices,
    'U.S. Stocks': ft.partial(si.get_tickers, 'SPX+DJIA+NDX+RUI+RUT+SOX'),
    'Taiwan Stocks': ft.partial(si.get_tickers, 'TWSE+TPEX+ESB'),
}[source]()
dropdowns, layout = create_search_dropdowns(symbols, 1)

period_dropdown2 = create_period_dropdown('2y')
interval_dropown2 = create_interval_dropdown('1wk')
stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'simple_white'),
}[backend]

btn_plot_prc = widgets.Button(description="Price/RS/Volume Chart",
                              layout=widgets.Layout(width='168px'))
btn_plot_fin = widgets.Button(description="Financial Chart",
                              layout=widgets.Layout(width='168px'))
btn_report_q = widgets.Button(description="Quarterly Report",
                              layout=widgets.Layout(width='168px'))
btn_report_a = widgets.Button(description="Annual Report",
                              layout=widgets.Layout(width='168px'))
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
outputs = widgets.VBox()

display(
    layout,
    period_dropdown2, interval_dropown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_report_q, btn_report_a]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

stock_chart = {
    'mplfinance': mpl_mansfield.StockChart,
    'Plotly': ply_mansfield.StockChart,
}[backend]

def on_plot_prc_click(b):
    selected = get_dropdowns_selected_options(dropdowns)
    new_output = widgets.Output()
    with new_output:
        interval = interval_dropown2.value
        period = period_dropdown2.value
        if not selected:
            print("No Stock Selected!")
        elif stock_chart is mpl_mansfield.StockChart:
            stock_chart.plot(selected[0], interval=interval, period=period,
                             style=stock_theme_dropdown.value,
                             legend_loc='upper left')
        else: # Plotly
            stock_chart.plot(selected[0], interval=interval, period=period,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = get_dropdowns_selected_options(dropdowns)
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        elif fin_chart is mpl_fin:
            fin_chart.plot(selected[0], style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected[0], template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_q_click(b):
    selected = get_dropdowns_selected_options(dropdowns)
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
    outputs.children = (new_output,) + outputs.children


def on_report_a_click(b):
    selected = get_dropdowns_selected_options(dropdowns)
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children



def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_report_q.on_click(on_report_q_click)
btn_report_a.on_click(on_report_a_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


Dropdown(description='Period:', index=1, options=('1y', '2y', '5y'), value='2y')

Dropdown(description='Interval:', index=1, options=('1d', '1wk'), value='1wk')

Dropdown(description='Stock Theme:', index=2, options=('default', 'classic', 'yahoo', 'charles', 'tradingview'…

VBox()